In [158]:
chain1 = next(iter(model1.get_chains()))
residues1 = list(chain1.get_residues())

chain2 = next(iter(model2.get_chains()))
residues2 = list(chain2.get_residues())[1:]

for i,(r1, r2) in enumerate(zip(residues1, residues2)):
    if r1.resname != r2.resname:
        print(f"Residue {i}: {r1.resname} != {r2.resname}")

Residue 0: MET != LYS
Residue 1: LYS != GLU
Residue 2: GLU != PHE
Residue 3: PHE != TRP
Residue 4: TRP != ASN
Residue 5: ASN != LEU
Residue 6: LEU != ASP
Residue 7: ASP != LYS
Residue 8: LYS != ASN
Residue 9: ASN != LEU
Residue 10: LEU != GLN
Residue 11: GLN != LEU
Residue 12: LEU != ARG
Residue 13: ARG != LEU
Residue 14: LEU != GLY
Residue 15: GLY != ILE
Residue 16: ILE != VAL
Residue 17: VAL != PHE
Residue 18: PHE != LEU
Residue 19: LEU != GLY
Residue 20: GLY != ALA
Residue 21: ALA != PHE
Residue 22: PHE != SER
Residue 23: SER != TYR
Residue 24: TYR != GLY
Residue 25: GLY != THR
Residue 26: THR != VAL
Residue 27: VAL != PHE
Residue 28: PHE != SER
Residue 30: SER != MET
Residue 31: MET != THR
Residue 32: THR != ILE
Residue 33: ILE != TYR
Residue 35: TYR != ASN
Residue 36: ASN != GLN
Residue 37: GLN != TYR
Residue 38: TYR != LEU
Residue 39: LEU != GLY
Residue 40: GLY != SER
Residue 41: SER != ALA
Residue 42: ALA != ILE
Residue 43: ILE != THR
Residue 44: THR != GLY
Residue 45: GLY != IL

In [155]:
residues1 = list(model1.get_chains())[0]
residues2 = list(model2.get_chains())[0]
print(len([r1.resname for r1 in residues1]), [r1.resname for r1 in residues1])
print(len([r2.resname for r2 in residues2]), [r2.resname for r2 in residues2])

408 ['MET', 'LYS', 'GLU', 'PHE', 'TRP', 'ASN', 'LEU', 'ASP', 'LYS', 'ASN', 'LEU', 'GLN', 'LEU', 'ARG', 'LEU', 'GLY', 'ILE', 'VAL', 'PHE', 'LEU', 'GLY', 'ALA', 'PHE', 'SER', 'TYR', 'GLY', 'THR', 'VAL', 'PHE', 'SER', 'SER', 'MET', 'THR', 'ILE', 'TYR', 'TYR', 'ASN', 'GLN', 'TYR', 'LEU', 'GLY', 'SER', 'ALA', 'ILE', 'THR', 'GLY', 'ILE', 'LEU', 'LEU', 'ALA', 'LEU', 'SER', 'ALA', 'VAL', 'ALA', 'THR', 'PHE', 'VAL', 'ALA', 'GLY', 'ILE', 'LEU', 'ALA', 'GLY', 'PHE', 'PHE', 'ALA', 'ASP', 'ARG', 'ASN', 'GLY', 'ARG', 'LYS', 'PRO', 'VAL', 'MET', 'VAL', 'PHE', 'GLY', 'THR', 'ILE', 'ILE', 'GLN', 'LEU', 'LEU', 'GLY', 'ALA', 'ALA', 'LEU', 'ALA', 'ILE', 'ALA', 'SER', 'ASN', 'LEU', 'PRO', 'GLY', 'HIS', 'VAL', 'ASN', 'PRO', 'TRP', 'SER', 'THR', 'PHE', 'ILE', 'ALA', 'PHE', 'LEU', 'LEU', 'ILE', 'SER', 'PHE', 'GLY', 'TYR', 'ASN', 'PHE', 'VAL', 'ILE', 'THR', 'ALA', 'GLY', 'ASN', 'ALA', 'MET', 'ILE', 'ILE', 'ASP', 'ALA', 'SER', 'ASN', 'ALA', 'GLU', 'ASN', 'ARG', 'LYS', 'VAL', 'VAL', 'PHE', 'MET', 'LEU', 'ASP', '

In [157]:
# compute rmsd between two protein structures

import sys
import numpy as np
from Bio.PDB import PDBParser, Superimposer

pdb1 = 'target.pdb'
pdb2 = 'casp-predictions/T1024/T1024TS013_2' # lowest rms_all from casp14

parser = PDBParser()
structure1 = parser.get_structure('1', pdb1)
structure2 = parser.get_structure('2', pdb2)

model1 = structure1[0]
model2 = structure2[0]

def get_backbone_atoms(model1, model2):
    atoms1 = []
    atoms2 = []
    for chain1, chain2 in zip(model1.get_chains(), model2.get_chains()):
        for residue1, residue2 in zip(chain1, chain2):
            # for atom1, atom2 in zip(residue1, residue2):
            #     atoms1.append(atom1)
            #     atoms2.append(atom2)
            for atom1 in residue1:
                if atom1.get_id() in ['CA']:
                # if atom1.get_id() in ['N', 'CA', 'C']:
                    atom2 = residue2[atom1.get_id()]
                    if atom2 is None:
                        raise ValueError('Atoms are not aligned')
                    atoms1.append(atom1)
                    atoms2.append(atom2)
    return np.array(atoms1), np.array(atoms2)

atoms1, atoms2 = get_backbone_atoms(model1, model2)
print(atoms1.shape, atoms2.shape)
sup = Superimposer()
sup.set_atoms(atoms1, atoms2)
sup.apply(atoms2) # align with SVD (PDB.SVDSuperimposer)

rmsd = sup.rms # equal to sqrt(sum(sum(diff * diff)) / coords1.shape[0]) where diff = coords1 - coords2

print(rmsd)

(408,) (408,)
21.05145397124449


/home/mmazeem/.local/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'N' for Atom (name=N) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/mmazeem/.local/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=CA) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/mmazeem/.local/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=C) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/mmazeem/.local/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'O' for Atom (name=O) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/mmazeem/.local/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'C' for Atom (name=CB) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/home/mmazeem

In [ ]:
# Compute backbone RMSD
# rmsd = rmsd(pdb1, pdb2, 'CA')
# print(rmsd)